In [1]:
# Load the TensorBoard notebook extension

#Uncomment to use it in colab and monitor to tendorboard
#%load_ext tensorboard

import tensorflow as tf
import datetime

##Data preparation

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import kerastuner as kt
from contextlib import redirect_stdout
import shutil
import locale

locale.setlocale(locale.LC_ALL, 'en_US.UTF-8') #This is needed to convert string values of type 1,999.99 
                                               #to float 1999.99


'en_US.UTF-8'

Define the path to load all locally stored data:

In [19]:
#Define where are the datasets

#path = os.getcwd()+"/exper_files/datasets/" #Here they are datasets from yahoo
path = os.getcwd()+"/exper_files/coinmarket/" #Here they are datasets from coinmarket


Create the whole dataset: 

In [24]:
def dataset_creation(crypto_list, pth):
    datasets = {} #Store all datasets here
    min_dates = {} #Store all minimum dates here
    max_dates = {} #Store all maximum dates here
    datasets_list = os.listdir(pth) #Find all dataset
    
    splitted_path = path.split('/')
    if splitted_path[-2] == 'coinmarket':
        type_of_file = '.txt'
    
    elif splitted_path[-2] == 'datasets':
        type_of_file = '.csv'
        
    
    crypto_list = [elem+type_of_file for elem in crypto_list]

    for dataset in datasets_list:
        if dataset.endswith(type_of_file) and (dataset) in crypto_list:

            name = dataset.split(".")
            dataset_name = name[0]
            
            if splitted_path[-2] == 'datasets':
                
                datasets[dataset_name] = pd.read_csv(pth + dataset) 

                datasets[dataset_name]['Date'] = pd.to_datetime(datasets[dataset_name]['Date']) #Dataset from yahoo
                                                                                            #is stored ia .csv file.
                    
                datasets[dataset].rename(columns={'Adj Close':dataset+'Adj_Close'}) #Space will cause troubles
                
            
            elif splitted_path[-2] == 'coinmarket':  
                    
                lines = []
                with open(pth + dataset) as f: #Dataset from coinmarket is stored in a .txt file, so we need to read it
                                               #line by line.
                    lines = f.readlines()
                
                initial_columns = lines[0].split("\t") #Split the first line to individual words. The first line 
                                                       #contains the column names. 
                columns = []
                for col in initial_columns:
                    clear_col = col.split('*') #Some column names contains * or **, so we need to exclude them
                    clear_col = clear_col[0].split('\n') #The last element cointains a \n
                    
                    if clear_col[0] == 'Market Cap':
                        clear_col[0] = 'Market_Cap' #Space will cause troubles
                    columns.append(clear_col[0])
                
                df = pd.DataFrame(columns=columns) #Define column names
                
                
                for line in range(1, len(lines)):
                    
                    splitted_line = lines[line].split("\t") #Split each line to individual numbers
                    
                    for elem in range(len(splitted_line)):
                        clear_elem = splitted_line[elem].split('$') #All number cointains a $ (except Date) so we need 
                                                                    #throw it away.
                        
                        if len(clear_elem) == 1:
                            splitted_line[elem] = pd.to_datetime(clear_elem[0])#Tranform each Date to 'datetime' type
                            
                        else:
                            clear_elem[1] = clear_elem[1].split('\n')
                            splitted_line[elem] = locale.atof(clear_elem[1][0])
                    
                    #Append each line to the dataframe
                    row_to_append = pd.Series(splitted_line, index=columns)                    
                    df = df.append(row_to_append, ignore_index=True)
                
                #Invert the sorting of values to ascending based on 'Date'
                df = df.sort_values(by=['Date']).reset_index().drop(['index'], axis=1)
                datasets[dataset_name] = df.copy()
            
            #There are a few missing values, so let fill them with the previous value
            datasets[dataset_name].fillna(method='ffill', inplace=True)

            #Create new columns 'close_off_high' and 'volatility' in order to make predictions more accurate:
            kwards = {'close_off_high': lambda x: 2 * (x['High'] - x['Close']) / (x['High'] - x['Low']) - 1,
              'volatility': lambda x: (x['High'] - x['Low']) / (x['Open'])
              }

            datasets[dataset_name] = datasets[dataset_name].assign(**kwards)
            
            datasets[dataset_name].fillna(-1, inplace=True) #Fill possible NaN close_off_high values with -1
                                                            #Because NaN values caused by (0/0)-1
            

            first_date = pd.to_datetime(datasets[dataset_name]['Date'][0])
            min_dates[dataset_name]=first_date
            
            last_date = pd.to_datetime(datasets[dataset_name]['Date'].iloc[-1])
            max_dates[dataset_name] = last_date
            

    max_date = max(min_dates.values(), key=lambda v: v)
    min_date = min(max_dates.values(), key=lambda v: v)

    #Drop all the data which are prior to max_date and later to min_date
    for dataset in datasets:
        datasets[dataset] = datasets[dataset][(datasets[dataset]['Date'] >= max_date) & 
                                              (datasets[dataset]['Date'] <= min_date)]


    #Compute the average and standard deviation of 'Close' value for the last 7-days and 30-days(month): 
    for dataset in datasets:

        temp = datasets[dataset].copy()

        #Drop the first 30 days to be able to compute average and standard deviation of month for the rows of the table
        temp = temp[29:]

        temp['mean_7days_Close'] = datasets[dataset]['Close'].rolling(window=7).mean()

        temp['mean_month_Close'] = datasets[dataset]['Close'].rolling(window=30).mean()

        temp['std_7days_Close'] = datasets[dataset]['Close'].rolling(window=7).std()

        temp['std_month_Close'] = datasets[dataset]['Close'].rolling(window=30).std()

        datasets[dataset] = temp.copy()
        



    #Rename the columns and concatenate the all datasets to one 
    count = 0
    for dataset in datasets:

        datasets[dataset] = datasets[dataset].rename(columns=lambda x: dataset+'_'+x)
        datasets[dataset] = datasets[dataset].rename(columns={dataset+'_Date': 'Date'})                                                   
        
        if count == 0:
            
            date_col = (datasets[dataset]['Date'].reset_index()).drop(['index'], axis=1)
            
            
        datasets[dataset] = ((datasets[dataset].drop(['Date'], axis=1)).reset_index()).drop(['index'], axis=1)    
                    
        

    whole_dataset = pd.concat([datasets[dataset] for dataset in datasets], axis=1)
    whole_dataset = pd.concat([date_col, whole_dataset], axis=1)
    
   
    return whole_dataset

Split data into training, validation and test set:

In [5]:
def split_data(perc_train_set, perc_val_set, currency_data):
        
        #Compute the date to split the dataset into training and validation_test set based on 'perc_train_set'
        splt_date_train = currency_data.iloc[round(currency_data.shape[0] * perc_train_set)]['Date']
        
        #Split the dataset into trainning and validation_test set
        tr_set, val_tst_set = currency_data[currency_data['Date'] < splt_date_train], \
                         currency_data[currency_data['Date'] >= splt_date_train]
        
        #Compute the date to split the val_tst_set into validation and test set based on 'perc_val_set'
        splt_date_val = val_tst_set.iloc[round(val_tst_set.shape[0] * perc_val_set)]['Date']

        #Split the val_tst_set into validation and test set        
        val_set, tst_set = val_tst_set[val_tst_set['Date'] < splt_date_val], \
                            val_tst_set[val_tst_set['Date'] >= splt_date_val]
        
        return tr_set, val_set, tst_set, splt_date_train, splt_date_val

Normalize training, validation and test inputs and outputs with sliding window:

In [6]:
def normalize_in_out(prd_range, wind_len, tr_set, val_set, tst_set, feats, coin_targ, crypto_list):
    
    all_feats = tr_set.columns #Get all features
    feats = [crypto+"_"+feat for crypto in crypto_list for feat in feats] #Get the features in the appropriate format 
                                                                          #(e.g 'Close' --> 'BTC-USD_Close')
    
    #Normalize training inputs
    LSTM_tr_in = []
    for i in range(len(tr_set) - wind_len):
        tmp_set = tr_set[i:(i + wind_len)].copy()
        
        for col in all_feats:
            if col not in feats:
                tmp_set = tmp_set.drop([col], axis=1) #Drop the feature that will not be used

        for col in feats:
            tmp_set[:][col] = tmp_set[col] / tmp_set[col].iloc[0] - 1 #Normalize the feature that will be used

        LSTM_tr_in.append(tmp_set)
    
    #Transform from DataFrame to numpy array
    LSTM_tr_in = [np.array(LSTM_tr_i) for LSTM_tr_i in LSTM_tr_in]
    LSTM_tr_in = np.array(LSTM_tr_in)
    
    
    #Normalize validation inputs
    LSTM_val_in = []
    for i in range(len(val_set) - wind_len):
        tmp_set = val_set[i:(i + wind_len)].copy()
        
        for col in all_feats:
            if col not in feats:
                tmp_set = tmp_set.drop([col], axis=1) #Drop the feature that will not be used

        for col in feats:
            tmp_set[:][col] = tmp_set[col] / tmp_set[col].iloc[0] - 1 #Normalize the feature that will be used
    
        LSTM_val_in.append(tmp_set)
        
    #Transform from DataFrame to numpy array
    LSTM_val_in = [np.array(LSTM_val_i) for LSTM_val_i in LSTM_val_in]
    LSTM_val_in = np.array(LSTM_val_in)
    
    
    #Normalize test inputs
    LSTM_test_in = []
    for i in range(len(tst_set) - wind_len):
        tmp_set = tst_set[i:(i + wind_len)].copy() 
        
        for col in all_feats:
            if col not in feats:
                
                tmp_set = tmp_set.drop([col], axis=1) #Drop the feature that will not be used

        for col in feats:
            tmp_set[:][col] = tmp_set[col] / tmp_set[col].iloc[0] - 1 #Normalize the feature that will be used

        LSTM_test_in.append(tmp_set)
    
    
    #Transform from DataFrame to numpy array
    LSTM_test_in = [np.array(LSTM_test_i) for LSTM_test_i in LSTM_test_in]
    LSTM_test_in = np.array(LSTM_test_in)
    
    
    #Normalize training outputs
    LSTM_rangd_train_out = []
    for i in range(wind_len, len(tr_set[coin_targ+'_Close']) - prd_range):
        LSTM_rangd_train_out.append((tr_set[coin_targ+'_Close'][i:i+prd_range].values/tr_set[coin_targ+'_Close'][tr_set.index[0]+i-wind_len]) - 1)

    LSTM_rangd_train_out = np.array(LSTM_rangd_train_out)
    
    
    #Normalize validation outputs
    LSTM_rangd_val_out = []
    for i in range(wind_len, len(val_set[coin_targ+'_Close']) - prd_range):
        LSTM_rangd_val_out.append((val_set[coin_targ+'_Close'][i:i+prd_range].values/val_set[coin_targ+'_Close'][val_set.index[0]+i-wind_len]) - 1)
    
    LSTM_rangd_val_out = np.array(LSTM_rangd_val_out)
    
    return LSTM_rangd_train_out, LSTM_rangd_val_out, LSTM_tr_in, LSTM_val_in, LSTM_test_in

Define model

In [7]:
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout
import tensorflow as tf


def build_model(inputs, output_size, neurons, activ_func="linear",
                dropout=0.25, loss="mae", optimizer="adam"):
    model = Sequential()
    model.add(LSTM(neurons, return_sequences=True, input_shape=(inputs.shape[1], inputs.shape[2])))
    
    model.add(Dropout(dropout))
    model.add(LSTM(units=neurons, return_sequences=True))
    
    model.add(Dropout(dropout))
    model.add(LSTM(units=neurons))
    
    model.add(Dropout(dropout))
    model.add(Dense(units=output_size))
    
    model.add(Activation(activ_func))

    model.compile(loss=loss, optimizer=optimizer)
    return model

Bulid and train model:

In [8]:
def build_and_train_model(epchs, btch_size, neurs, dropout, prd_range, LSTM_train_in, LSTM_rangd_train_out, LSTM_valid_in, LSTM_rangd_valid_out, shffl, verb, early_st_pat):#, tbd_ck):

    rnged_btcoin_model = build_model(LSTM_train_in, output_size=prd_range, neurons=neurs, dropout=dropout)
    
    np.random.seed(202)
    
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=early_st_pat)]#, tbd_ck] #Uncomment to 
                                                                                                        #enable tensorboard

    rnged_hist = rnged_btcoin_model.fit(LSTM_train_in[:-prd_range], LSTM_rangd_train_out,
                                      validation_data=(LSTM_valid_in[:-prd_range], LSTM_rangd_valid_out), 
                                        epochs=epchs, batch_size=btch_size, verbose=verb,  callbacks=callbacks, 
                                        shuffle=shffl)
    
    return rnged_btcoin_model, rnged_hist

Create MAE plot:

In [52]:
def create_plot(rangd_h, path_to_sav, target_cn, loss='loss'):
    
    fig, ax1 = plt.subplots(1, 1, figsize=(25, 10))

    ax1.plot(rangd_h.epoch, rangd_h.history[loss], color='b', label='Loss')
    ax1.plot(rangd_h.epoch, rangd_h.history['val_'+loss], color='orange', label='Val_loss')
    ax1.set_xticks(range(0, len(rangd_h.epoch)))
    ax1.set_title('Training Error')
    ax1.set_ylabel('Mean Absolute Error (MAE)', fontsize=12)
    ax1.set_xlabel('#Epoch', fontsize=12)
    plt.legend()
    plt.savefig(path_to_sav+"/"+target_cn+'_MAE.png')
    plt.close(fig)
    #plt.show()

In [10]:
#Get data
crypto_list = [['BTC-USD', 'ETH-USD']] #[['BTC-USD', 'ETH-USD'], ['BTC-USD']] #Define the sets of cryptocurrencies to be tested

first_txt_flag = 0

best_perf = np.inf
best_perf_details = ""


for cryptocurrency_list in crypto_list:
    
    cryptocurrency_list_output = 'Using cryptocurrencies: '+str(cryptocurrency_list)
    print(cryptocurrency_list_output)
    data = dataset_creation(cryptocurrency_list, path)

    #Split data
    percent_train_set = 0.8
    percent_val_set = 0.5

    training_set, validation_set, test_set, split_date_train, split_date_valid = split_data(percent_train_set, 
                                                                                           percent_val_set, data)

    
    features_list = [['Close', 'Open', 'High']]
    """[['Close'], ['Close', 'Volume'], ['Close', 'Open', 'High'],
                     ['Close', 'close_off_high', 'volatility'],
                     ['Close', 'mean_7days_Close', 'mean_month_Close'],
                     ['Close', 'std_7days_Close', 'std_month_Close']]""" #Define the sets of features to be tested 
    
    for featurs in features_list: 
        
        new_txt_flag = 0 
        
        featurs_output = '\tUsing the features: '+str(featurs)
        print('\tUsing the features: '+str(featurs))
        
        
        #Create inputs and outputs for the model training, validation and testing
        pred_range = 5
        window_len = 10
        features = featurs
        coin_target = 'BTC-USD'
        
        

        LSTM_ranged_training_outputs, LSTM_ranged_validation_outputs, LSTM_training_inputs, LSTM_validation_inputs, LSTM_test_inputs = normalize_in_out(
                                                                                                      pred_range, window_len, 
                                                                                                      training_set, 
                                                                                                      validation_set, 
                                                                                                      test_set, 
                                                                                                      features, coin_target,
                                                                                                      cryptocurrency_list)

        batch_size_list = [1, 32, 64]
        neuron_list = [20, 40, 60, 100]
        dropout_list = [0.2, 0.25, 0.3, 0.4]

        for bat_s, neur, drop in [(bat_s, neur, drop) for bat_s in batch_size_list for neur in neuron_list for drop in dropout_list]:

            #Build and train model
            epochs = 100
            batch_size = bat_s
            neurons = neur
            dropout = drop
            early_stop_patience = 10
            shuffle = True
            verbose = 0
            
            exper_params_output = '\t\tBatch_size: '+str(batch_size)+" Neurons: "+str(neurons)+" Dropout: "+str(dropout)
            print(exper_params_output)
           
            
            
            #Path to write log files
            cur_path = os.getcwd()+'/'
            log_dir = cur_path + "logs/fit/" + str(cryptocurrency_list) + '/' + str(featurs) + '/' + 'Batch_size='+str(batch_size+"_Neurons="+str(neurons)+"_Dropout="+str(dropout))
            if not os.path.exists(log_dir):
                os.makedirs(log_dir)

                            
                            
            #Unocomment to monitor in tensorboard
            #tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
            
           
            
            ranged_btcoin_model, ranged_hist = build_and_train_model(epochs, batch_size, neurons, dropout, pred_range, 
                                                                     LSTM_training_inputs, LSTM_ranged_training_outputs, 
                                                                     LSTM_validation_inputs, LSTM_ranged_validation_outputs, 
                                                                     shuffle, verbose, early_stop_patience)#, tensorboard_callback)

            exper_outputs = '\t\tBest model found in epoch ' + str(ranged_hist.epoch[-early_stop_patience]) +' with validation loss: '+str(ranged_hist.history['val_loss'][-early_stop_patience])
            print(exper_outputs)
            
            if best_perf > ranged_hist.history['val_loss'][-early_stop_patience]:
                best_perf = ranged_hist.history['val_loss'][-early_stop_patience]
                best_perf_details = cryptocurrency_list_output +"\n"+featurs_output+"\n"+exper_params_output+ \
                                    "\n"+exper_outputs
            
            #Print results to a .txt file
            if first_txt_flag == 0:
                
                if new_txt_flag == 0:
                    
                    #Create a .txt file and write the results
                    txt_log_dir = "logs/fit/" + str(cryptocurrency_list) + '/' + str(featurs) + '/'
                    f = open(txt_log_dir+"/"+coin_target+"_results.txt", "w")
                    f.write(cryptocurrency_list_output)
                    f.write("\n"+featurs_output)
                    
                    first_txt_flag += 1
                    new_txt_flag += 1
            else:
                
                if new_txt_flag == 0:
                    
                    f.close() #Close previous txt file
                    
                    #Create a .txt file and write the results
                    txt_log_dir = "logs/fit/" + str(cryptocurrency_list) + '/' + str(featurs) + '/'
                    f = open(txt_log_dir+"/"+coin_target+"_results.txt", "w")
                    f.write(cryptocurrency_list_output)
                    f.write("\n"+featurs_output)
                    
                    new_txt_flag += 1
                    
                
            
            f.write("\n"+exper_params_output)
            f.write("\n"+exper_outputs)
            
            create_plot(ranged_hist, log_dir, coin_target)

f.close()

f_best = open("logs/fit/"+coin_target+"_best_results.txt", "w")
f_best.write(best_perf_details)
f_best.close()

SyntaxError: invalid syntax (<ipython-input-10-4590d4532fa6>, line 87)

In [ ]:
#Uncomment to monitor in tensorboard

#%tensorboard dev list
#%tensorboard dev upload --logdir ./logs

#%tensorboard dev delete --experiment_id EXPERIMENT ID HERE

In [ ]:
#ranged_btcoin_model.summary()

It seems that more than one cryptocurrencies dataset lead to better results. Moreover, features ['Close', 'Open', 'High'] return the lower losses. Finally, concerning 'batch_size', number 64 are resulted to the more efficient outcomes. Thus, let use these parameters to hypertune our models to be tested (for the rest parameters):

In [115]:
########Model with LSTM layers followed by Dense layers #####################

def model_tuning(inputs, outputs, output_size, validation_inputs, validation_outputs, epochs, early_stop_patience, 
                 neurons, dropout, prd_range, batch_size, keras_tuner_dir, activ_func="linear", loss="mae", 
                 optimizer="adam"):
    
    def build_model(hp):
        model = Sequential()

        neurLSTM1 = hp.Choice('neurons_1_LSTM', values=neurons)
        model.add(LSTM(neurLSTM1, input_shape=(inputs.shape[1], inputs.shape[2]))) #return_sequences=True

        dropLSTM1 = hp.Choice('dropout_LSTM_1', values=dropout)
        model.add(Dropout(dropLSTM1))
        
        #neurLSTM2 = hp.Choice('neurons_2_LSTM', values=neurons)
        #model.add(LSTM(neurLSTM2))
        
        #dropLSTM2 = hp.Choice('dropout_LSTM_2', values=dropout)
        #model.add(Dropout(dropLSTM2))
        

        neurD1 = hp.Choice('neurons_1_Dense', values=[32, 64, 128, 256])
        model.add(Dense(units=neurD1, activation='relu'))
        
        #dropDense1 = hp.Choice('dropout_Dense_1', values=dropout)
        #model.add(Dropout(dropDense1))
        
        #neurD2 = hp.Choice('neurons_2_Dense', values=[32, 64, 128, 256])
        #model.add(Dense(units=neurD2, activation='relu'))
        
        #dropDense2 = hp.Choice('dropout_Dense_2', values=dropout)
        #model.add(Dropout(dropDense2))
                  
        model.add(Dense(units=output_size))
        model.add(Activation(activ_func))
        
        
        #RMSProp optimization
        hp_learning_rate = hp.Float(name='learning_rate', min_value=0.0001, max_value=0.05, step=0.0005)
        hp_rho = hp.Float(name='rho', min_value=0.8, max_value=0.95, step=0.01)
        hp_momentum = hp.Float(name='momentum', min_value=0.0, max_value=0.00001, step=0.000001)
        hp_epsilon = hp.Float(name='epsilon', min_value=1e-08, max_value=1e-07, step=1e-08)
        
        optimizer = tf.keras.optimizers.RMSprop(
        learning_rate=hp_learning_rate, rho=hp_rho, momentum=hp_momentum, epsilon=hp_epsilon
        )
        
        """
        
        #Adam optimization
        
        hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
        
        hp_beta_1 = hp.Choice('beta_1', values=[0.85, 0.9, 0.95])
        
        hp_beta_2 = hp.Choice('beta_2', values=[0.999, 0.995, 0.98])
        
        hp_epsilon = hp.Choice('epsilon', values=[1e-07, 1e-08])
        
        optimizer = tf.keras.optimizers.Adam(
        learning_rate=hp_learning_rate, beta_1=hp_beta_1, beta_2=hp_beta_2, epsilon=hp_epsilon)"""
        
        model.compile(loss=loss, optimizer=optimizer)
        return model
    
    
    
    #Create_tuner
    tuner = kt.Hyperband(build_model,
                 objective='val_loss',
                 max_epochs=epochs,
                 factor=3,
                 directory=keras_tuner_dir,
                 project_name='keras_tuner')

    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=early_stop_patience)

    tuner.search(inputs[:-prd_range], outputs, 
                 validation_data=(validation_inputs[:-prd_range], validation_outputs), 
                 epochs=epochs, batch_size=batch_size,callbacks=[stop_early],
                 use_multiprocessing=True,
                 workers=8)

    # Get the optimal hyperparameters
    best_hps=tuner.get_best_hyperparameters(num_trials=3)[0]

    
    return best_hps, tuner
    

In [59]:
#Get data
crypto_list = [['BTC-USD', 'ETH-USD', 'DOGE-USD', 'USDT-USD']]#, 'ETH-USD', 'DOGE-USD', 'USDT-USD']] #,'XRP-USD', 'LTC-USD', 'XLM-USD', 
                #'XMR-USD', 'BNB-USD', 'ADA-USD', 'BCH-USD', 'LINK-USD', 'ETC-USD', 'EOS-USD', 'TRX-USD',
               #'FIL-USD', 'NEO-USD', 'XTZ-USD', 'MIOTA-USD']] #Define the sets of cryptocurrencies to be tested

first_txt_flag = 0


for cryptocurrency_list in crypto_list:
    
    cryptocurrency_list_output = 'Using cryptocurrencies: '+str(cryptocurrency_list)
    print(cryptocurrency_list_output)
    data = dataset_creation(cryptocurrency_list, path)

    #Split data
    percent_train_set = 0.8
    percent_val_set = 0.5

    training_set, validation_set, test_set, split_date_train, split_date_valid = split_data(percent_train_set, 
                                                                                           percent_val_set, data)

    
    features_list = [['Close', 'Open', 'High']]
    """[['Close'], ['Close', 'Volume'], ['Close', 'Open', 'High'],
                     ['Close', 'close_off_high', 'volatility'],
                     ['Close', 'mean_7days_Close', 'mean_month_Close'],
                     ['Close', 'std_7days_Close', 'std_month_Close']]""" #Define the sets of features to be tested 
    
    for featurs in features_list: 
        
        new_txt_flag = 0 
        
        featurs_output = '\tUsing the features: '+str(featurs)
        print('\tUsing the features: '+str(featurs))
        
        
        #Create inputs and outputs for the model training, validation and testing
        pred_range = 5
        window_len = 10
        features = featurs
        coin_target = 'BTC-USD'
        
        

        LSTM_ranged_training_outputs, LSTM_ranged_validation_outputs, LSTM_training_inputs, LSTM_validation_inputs, LSTM_test_inputs = normalize_in_out(
                                                                                                      pred_range, window_len, 
                                                                                                      training_set, 
                                                                                                      validation_set, 
                                                                                                      test_set, 
                                                                                                      features, coin_target,
                                                                                                      cryptocurrency_list)

        batch_size_list = [64] #[1, 32, 64]
        neuron_list = [20, 40, 60, 100]
        dropout_list = [0.0, 0.1, 0.2, 0.25, 0.3, 0.4]
        early_stop_patience = 10
        epochs = 500
        shuffle = True
        verbose = 0
        
        
        
        
        for bat_s in batch_size_list:

            #Build and train model
            batch_size = bat_s
            
            
            exper_params_output = '\t\tBatch_size: '+str(batch_size)
            print(exper_params_output)
           
            
            #Path to write log files
            cur_path = os.getcwd()+'/'
            log_dir = cur_path + "logs/fit/" + str(cryptocurrency_list) + '/' + str(featurs) + '/' + 'Batch_size='+str(batch_size)
            if not os.path.exists(log_dir):
                os.makedirs(log_dir)
            
            #Path to save its checkpoints the keras_tuner because it raises an error when it is in the current working
            #directory or in a sub-folder
            keras_tuner_dir = 'C:\keras_tuner'
            if not os.path.exists(keras_tuner_dir):
                os.makedirs(keras_tuner_dir)
            
            best_hps, tuner = model_tuning(inputs=LSTM_training_inputs, outputs=LSTM_ranged_training_outputs,
                                output_size=pred_range, 
                                validation_inputs = LSTM_validation_inputs, 
                                validation_outputs = LSTM_ranged_validation_outputs,
                                epochs=epochs, early_stop_patience=early_stop_patience, neurons=neuron_list, 
                                dropout=dropout_list, prd_range=pred_range, batch_size=bat_s, 
                                           keras_tuner_dir=keras_tuner_dir)
        
            exper_outputs = "\t\tThe hyperparameter search is complete. The optimal parameter were found to be: \n" + \
                            str(tuner.oracle.get_best_trials(num_trials=1)[0].hyperparameters.values)
        
            
            
            # Build the model with the optimal hyperparameters and train it
            best_model = tuner.hypermodel.build(best_hps)
            mod_history = best_model.fit(LSTM_training_inputs[:-pred_range], LSTM_ranged_training_outputs, 
                                         validation_data=(LSTM_validation_inputs[:-pred_range], 
                                                          LSTM_ranged_validation_outputs), 
                                         epochs=epochs)
            
            #Get the epoch with the best validation loss
            val_loss_per_epoch = mod_history.history['val_loss']
            best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
            exper_outputs += '\n\n\t\tBest epoch: '+ str(best_epoch) + ' with validation loss: ' + str((min(val_loss_per_epoch)))
            
            #Print results to a .txt file
            if first_txt_flag == 0:
                
                if new_txt_flag == 0:
                    
                    #Create a .txt file and write the results
                    txt_log_dir = "logs/fit/" + str(cryptocurrency_list) + '/' + str(featurs) + '/'
                    f = open(txt_log_dir+"/"+coin_target+"_results.txt", "w")
                    f.write(cryptocurrency_list_output)
                    f.write("\n"+featurs_output)
                    
                    first_txt_flag += 1
                    new_txt_flag += 1
            else:
                
                if new_txt_flag == 0:
                    
                    f.close() #Close previous txt file
                    
                    #Create a .txt file and write the results
                    txt_log_dir = "logs/fit/" + str(cryptocurrency_list) + '/' + str(featurs) + '/'
                    f = open(txt_log_dir+"/"+coin_target+"_results.txt", "w")
                    f.write(cryptocurrency_list_output)
                    f.write("\n"+featurs_output)
                    
                    new_txt_flag += 1
                    
                
            
            f.write("\n"+exper_params_output)
            f.write("\n"+exper_outputs)
            
            f.write("\n\nFull keras tuner results: \n")
            with redirect_stdout(f): 
                tuner.results_summary()
            
            
            create_plot(mod_history, log_dir, coin_target)

f.close()


#Write best model summary to a txt file
with open(cur_path + 'logs/fit/modelsummary.txt', 'w') as f_sum:
    with redirect_stdout(f_sum):
        best_model.summary()
        

#Remove folder with is created by keras tuner
shutil.rmtree(keras_tuner_dir)

Trial 725 Complete [00h 00m 08s]
val_loss: 0.0023630305659025908

Best val_loss So Far: 0.0021931170485913754
Total elapsed time: 00h 56m 04s
INFO:tensorflow:Oracle triggered exit
Epoch 1/500
52/52 [==============================] - 3s 21ms/step - loss: 0.0139 - mae: 0.0840 - val_loss: 0.0027 - val_mae: 0.0373
Epoch 2/500
52/52 [==============================] - 0s 6ms/step - loss: 0.0079 - mae: 0.0624 - val_loss: 0.0030 - val_mae: 0.0408
Epoch 3/500
52/52 [==============================] - 0s 6ms/step - loss: 0.0064 - mae: 0.0562 - val_loss: 0.0023 - val_mae: 0.0343
Epoch 4/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0066 - mae: 0.0552 - val_loss: 0.0033 - val_mae: 0.0440
Epoch 5/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0065 - mae: 0.0564 - val_loss: 0.0024 - val_mae: 0.0345
Epoch 6/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0060 - mae: 0.0530 - val_loss: 0.0040 - val_mae: 0.0497
Epoch 7/500
52/52 [==============

Epoch 62/500
52/52 [==============================] - 0s 4ms/step - loss: 0.0044 - mae: 0.0481 - val_loss: 0.0041 - val_mae: 0.0513
Epoch 63/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0053 - mae: 0.0532 - val_loss: 0.0030 - val_mae: 0.0404
Epoch 64/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0045 - mae: 0.0477 - val_loss: 0.0028 - val_mae: 0.0384
Epoch 65/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0047 - mae: 0.0486 - val_loss: 0.0030 - val_mae: 0.0399
Epoch 66/500
52/52 [==============================] - 0s 4ms/step - loss: 0.0043 - mae: 0.0471 - val_loss: 0.0034 - val_mae: 0.0449
Epoch 67/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0046 - mae: 0.0489 - val_loss: 0.0028 - val_mae: 0.0379
Epoch 68/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0043 - mae: 0.0468 - val_loss: 0.0031 - val_mae: 0.0399
Epoch 69/500
52/52 [==============================] - 0s 5ms/step - loss: 0.

Epoch 124/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0036 - mae: 0.0432 - val_loss: 0.0037 - val_mae: 0.0443
Epoch 125/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0032 - mae: 0.0411 - val_loss: 0.0031 - val_mae: 0.0404
Epoch 126/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0036 - mae: 0.0422 - val_loss: 0.0042 - val_mae: 0.0484
Epoch 127/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0036 - mae: 0.0431 - val_loss: 0.0037 - val_mae: 0.0445
Epoch 128/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0035 - mae: 0.0431 - val_loss: 0.0033 - val_mae: 0.0416
Epoch 129/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0034 - mae: 0.0415 - val_loss: 0.0037 - val_mae: 0.0440
Epoch 130/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0037 - mae: 0.0430 - val_loss: 0.0039 - val_mae: 0.0439
Epoch 131/500
52/52 [==============================] - 0s 5ms/step - 

52/52 [==============================] - 0s 5ms/step - loss: 0.0027 - mae: 0.0385 - val_loss: 0.0041 - val_mae: 0.0476
Epoch 186/500
52/52 [==============================] - 0s 4ms/step - loss: 0.0025 - mae: 0.0366 - val_loss: 0.0038 - val_mae: 0.0452
Epoch 187/500
52/52 [==============================] - 0s 4ms/step - loss: 0.0029 - mae: 0.0381 - val_loss: 0.0035 - val_mae: 0.0427
Epoch 188/500
52/52 [==============================] - 0s 4ms/step - loss: 0.0030 - mae: 0.0391 - val_loss: 0.0038 - val_mae: 0.0458
Epoch 189/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0026 - mae: 0.0375 - val_loss: 0.0041 - val_mae: 0.0475
Epoch 190/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0033 - mae: 0.0406 - val_loss: 0.0043 - val_mae: 0.0497
Epoch 191/500
52/52 [==============================] - 0s 4ms/step - loss: 0.0028 - mae: 0.0385 - val_loss: 0.0037 - val_mae: 0.0459
Epoch 192/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0027 -

Epoch 247/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0027 - mae: 0.0376 - val_loss: 0.0037 - val_mae: 0.0471
Epoch 248/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0024 - mae: 0.0350 - val_loss: 0.0037 - val_mae: 0.0465
Epoch 249/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0025 - mae: 0.0363 - val_loss: 0.0036 - val_mae: 0.0452
Epoch 250/500
52/52 [==============================] - 0s 7ms/step - loss: 0.0024 - mae: 0.0351 - val_loss: 0.0040 - val_mae: 0.0493
Epoch 251/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0027 - mae: 0.0372 - val_loss: 0.0034 - val_mae: 0.0445
Epoch 252/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0024 - mae: 0.0349 - val_loss: 0.0043 - val_mae: 0.0506
Epoch 253/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0028 - mae: 0.0380 - val_loss: 0.0043 - val_mae: 0.0497
Epoch 254/500
52/52 [==============================] - 0s 5ms/step - 

52/52 [==============================] - 0s 5ms/step - loss: 0.0024 - mae: 0.0345 - val_loss: 0.0038 - val_mae: 0.0472
Epoch 309/500
52/52 [==============================] - 0s 4ms/step - loss: 0.0023 - mae: 0.0345 - val_loss: 0.0040 - val_mae: 0.0488
Epoch 310/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0022 - mae: 0.0339 - val_loss: 0.0040 - val_mae: 0.0493
Epoch 311/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0025 - mae: 0.0356 - val_loss: 0.0046 - val_mae: 0.0522
Epoch 312/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0022 - mae: 0.0338 - val_loss: 0.0038 - val_mae: 0.0473
Epoch 313/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0026 - mae: 0.0358 - val_loss: 0.0038 - val_mae: 0.0480
Epoch 314/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0022 - mae: 0.0337 - val_loss: 0.0039 - val_mae: 0.0476
Epoch 315/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0022 -

Epoch 370/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0021 - mae: 0.0325 - val_loss: 0.0039 - val_mae: 0.0477
Epoch 371/500
52/52 [==============================] - 0s 6ms/step - loss: 0.0020 - mae: 0.0324 - val_loss: 0.0037 - val_mae: 0.0473
Epoch 372/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0024 - mae: 0.0354 - val_loss: 0.0037 - val_mae: 0.0463
Epoch 373/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0021 - mae: 0.0335 - val_loss: 0.0038 - val_mae: 0.0473
Epoch 374/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0021 - mae: 0.0329 - val_loss: 0.0040 - val_mae: 0.0487
Epoch 375/500
52/52 [==============================] - 0s 6ms/step - loss: 0.0021 - mae: 0.0330 - val_loss: 0.0041 - val_mae: 0.0493
Epoch 376/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0023 - mae: 0.0345 - val_loss: 0.0035 - val_mae: 0.0460
Epoch 377/500
52/52 [==============================] - 0s 5ms/step - 

52/52 [==============================] - 0s 5ms/step - loss: 0.0022 - mae: 0.0329 - val_loss: 0.0038 - val_mae: 0.0476
Epoch 432/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0021 - mae: 0.0327 - val_loss: 0.0042 - val_mae: 0.0499
Epoch 433/500
52/52 [==============================] - 0s 4ms/step - loss: 0.0022 - mae: 0.0327 - val_loss: 0.0037 - val_mae: 0.0462
Epoch 434/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0021 - mae: 0.0321 - val_loss: 0.0038 - val_mae: 0.0474
Epoch 435/500
52/52 [==============================] - 0s 4ms/step - loss: 0.0023 - mae: 0.0340 - val_loss: 0.0040 - val_mae: 0.0482
Epoch 436/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0021 - mae: 0.0328 - val_loss: 0.0038 - val_mae: 0.0474
Epoch 437/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0020 - mae: 0.0318 - val_loss: 0.0039 - val_mae: 0.0486
Epoch 438/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0023 -

Epoch 493/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0022 - mae: 0.0335 - val_loss: 0.0039 - val_mae: 0.0487
Epoch 494/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0023 - mae: 0.0332 - val_loss: 0.0036 - val_mae: 0.0465
Epoch 495/500
52/52 [==============================] - 0s 5ms/step - loss: 0.0022 - mae: 0.0334 - val_loss: 0.0040 - val_mae: 0.0497
Epoch 496/500
52/52 [==============================] - 0s 6ms/step - loss: 0.0026 - mae: 0.0357 - val_loss: 0.0041 - val_mae: 0.0503
Epoch 497/500
52/52 [==============================] - 0s 8ms/step - loss: 0.0021 - mae: 0.0326 - val_loss: 0.0036 - val_mae: 0.0475
Epoch 498/500
52/52 [==============================] - 0s 7ms/step - loss: 0.0023 - mae: 0.0334 - val_loss: 0.0037 - val_mae: 0.0478
Epoch 499/500
52/52 [==============================] - 0s 6ms/step - loss: 0.0023 - mae: 0.0334 - val_loss: 0.0038 - val_mae: 0.0485
Epoch 500/500
52/52 [==============================] - 0s 5ms/step - 

In [58]:
########Model with CNN layers, followed by LSTM layers, followed by Dense layers #####################
from keras.layers import Conv1D
from keras.layers import MaxPool1D
from keras.layers import AveragePooling1D
from keras.layers import Conv2D
from keras.layers import MaxPool2D

from keras.layers import BatchNormalization
from keras.models import Sequential
from keras.layers import Activation, Dense
from keras.layers import LSTM
from keras.layers import Dropout
import tensorflow as tf


def model_tuning(inputs, outputs, output_size, validation_inputs, validation_outputs, epochs, early_stop_patience, 
                 neurons, dropout, prd_range, batch_size, keras_tuner_dir, activ_func="linear", loss="mae", 
                 optimizer="adam"):
    
    def build_model(hp):
        model = Sequential()
        
        
        ###########1D CONV############
        #filters_conv1 = hp.Choice('filters_Conv1', values=[16, 32, 64])
        #kernel_size_conv1 = hp.Choice('kernel_size_Conv1', values=[2, 3, 5])
        #strides_conv1 = hp.Choice('strides_Conv1', values=[1, 2, 3])
        #model.add(Conv1D(filters=filters_conv1, kernel_size=kernel_size_conv1, strides=strides_conv1, 
        #                 activation='relu',
        #                input_shape=(inputs.shape[1], inputs.shape[2])))
       
    
        #############2D CONV##########
        #filters_conv2 = hp.Choice('filters_Conv2', values=[16, 32, 64])
        #kernel_size_conv2 = hp.Choice('kernel_size_Conv2', values=[2, 3, 5])
        #strides_conv2 = hp.Choice('strides_Conv2', values=[(1, 1), (2, 2), (3, 3)])
        #tf.keras.layers.Conv2D(filters=filters_conv2, kernel_size=kernel_size_conv2, strides=strides_conv2, 
        #                       activation='relu',
        #                       input_shape=(1, inputs.shape[1], inputs.shape[2]), data_format=channels_first)
        
        
        #moment_batchNormConv1 = hp.Float('momentum_batchNormConv1', min_value=0.09, max_value=0.99, step=0.1)
        #model.add(BatchNormalization(momentum=moment_batchNormConv1))
        
        
        ######1D POOL########
        #pool_size_maxPool1 = hp.Choice('pool_size_MaxPool1', values=[2, 3, 4])
        #model.add(MaxPool1D(pool_size=2))
        #model.add(AveragePooling1D(pool_size=2))
        
        
        #############2D POOL##########
        #model.add(MaxPool2D(pool_size=(2, 2), data_format=channels_first)

        neurLSTM1 = hp.Choice('neurons_1_LSTM', values=neurons)
        model.add(LSTM(neurLSTM1, input_shape=(inputs.shape[1], inputs.shape[2]))) #return_sequences=True

        dropLSTM1 = hp.Choice('dropout_LSTM_1', values=dropout)
        model.add(Dropout(dropLSTM1))
        
        #neurLSTM2 = hp.Choice('neurons_2_LSTM', values=neurons)
        #model.add(LSTM(neurLSTM2))
        
        #dropLSTM2 = hp.Choice('dropout_LSTM_2', values=dropout)
        #model.add(Dropout(dropLSTM2))
        
        #moment_batchNormLSTM2 = hp.Float('momentum_batchNormLSTM2', min_value=0.09, max_value=0.99, step=0.1)
        #model.add(BatchNormalization(momentum=moment_batchNormLSTM2))

        neurD1 = hp.Choice('neurons_1_Dense', values=[32, 64, 128, 256])
        model.add(Dense(units=neurD1, activation='relu'))
        
        #dropDense1 = hp.Choice('dropout_Dense_1', values=dropout)
        #model.add(Dropout(dropDense1))
        
        #moment_batchNormDense1 = hp.Float('momentum_batchNormDense1', min_value=0.09, max_value=0.99, step=0.1)
        #model.add(BatchNormalization(momentum=moment_batchNormDense1))
        
        neurD2 = hp.Choice('neurons_2_Dense', values=[32, 64, 128, 256])
        model.add(Dense(units=neurD2, activation='relu'))
        
        #dropDense2 = hp.Choice('dropout_Dense_2', values=dropout)
        #model.add(Dropout(dropDense2))
        
        #moment_batchNormDense2 = hp.Float('momentum_batchNormDense2', min_value=0.09, max_value=0.99, step=0.1)
        #model.add(BatchNormalization(momentum=moment_batchNormDense2))
        
        model.add(Dense(units=output_size))
        model.add(Activation(activ_func))
        
        hp_learning_rate = hp.Float(name='learning_rate', min_value=0.0001, max_value=0.05, step=0.0005)
        
        hp_beta_1 = hp.Float('beta_1', min_value=0.85, max_value=0.95, step=0.01)
        
        hp_beta_2 = hp.Float('beta_2', min_value=0.98, max_value=0.999, step=0.001)
        
        hp_epsilon = hp.Float('epsilon', min_value=1e-07, max_value=1e-08, sampling='LOG')
        
        optimizer = tf.keras.optimizers.Adam(
        learning_rate=hp_learning_rate, beta_1=hp_beta_1, beta_2=hp_beta_2, epsilon=hp_epsilon)
        
        model.compile(loss='mse', optimizer=optimizer, metrics=['mae'])
        return model
    
    
    
    #Create_tuner
    tuner = kt.Hyperband(build_model,
                 objective='val_loss',
                 max_epochs=epochs,
                 factor=3,
                 directory=keras_tuner_dir,
                 project_name='keras_tuner')

    stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=early_stop_patience)

    tuner.search(inputs[:-prd_range], outputs, 
                 validation_data=(validation_inputs[:-prd_range], validation_outputs), 
                 epochs=epochs, batch_size=batch_size,callbacks=[stop_early],
                 use_multiprocessing=True,
                 workers=8)

    # Get the optimal hyperparameters
    best_hps=tuner.get_best_hyperparameters(num_trials=3)[0]

    
    return best_hps, tuner
    

In [60]:
create_plot(mod_history, log_dir, coin_target, loss='mae')